# **Clarifai as destination connector using Unstructured.IO**

This Notebook walk you through the step by step guide on how to utilise Clarifai as your destination connector using Unstructured.IO and import raw files from various source connectors. For the demo we will be using github as our source connector which has out raw text files.

### Setup
Install necessary libraries.

In [ ]:
! pip install "unstructured[clarifai]" #make sure the unstructured version is 0.13 or above

In [ ]:
!pip install "unstructured[github]" #since our source is git
!pip install httpx

## Github setup

For the demo we'll be using github repo of clarifai-python open-source repo https://github.com/Clarifai/clarifai-python

In [12]:
repo='Clarifai/clarifai-python' # Any repo (public) in the format of username/reponame
token='github_pat_*****'
from unstructured.ingest.connector.git import GitAccessConfig
from unstructured.ingest.connector.github import SimpleGitHubConfig
from unstructured.ingest.interfaces import PartitionConfig, ProcessorConfig, ReadConfig
from unstructured.ingest.runner import GithubRunner

## Clarifai and Github imports

In [13]:
import os
from unstructured.ingest.interfaces import (
    PartitionConfig,
    ProcessorConfig,
    ChunkingConfig,
    ReadConfig,
)

from unstructured.ingest.connector.clarifai import (
    ClarifaiAccessConfig,
    ClarifaiWriteConfig,
    SimpleClarifaiConfig,
)

from unstructured.ingest.runner.writers.base_writer import Writer
from unstructured.ingest.runner.writers.clarifai import (
    ClarifaiWriter,
)

## Clarifai_writer func which will configure the target clarifai app where the ingested documents will be loaded with initialising clarifai PAT as `api_key`

In [19]:
def clarifai_writer() -> Writer:
    return ClarifaiWriter(
        connector_config=SimpleClarifaiConfig(
            access_config=ClarifaiAccessConfig(
                api_key= "CLARIFAI_PAT"
            ),
            app_id= "YOUR_APP_NAME",
            user_id= "YOUR_USER_NAME"
            ),
        write_config=ClarifaiWriteConfig()
    )

## Let's now package the writer and runner together, make sure you have your S3 bucket URI ready.

In [ ]:
if __name__ == "__main__":
    writer = clarifai_writer()
    runner = GithubRunner(
        processor_config=ProcessorConfig(
            verbose=True,
            output_dir="github-ingest-output-local-folder",
            num_processes=2,
        ),
        read_config=ReadConfig(),
        partition_config=PartitionConfig(),
        connector_config=SimpleGitHubConfig(
            url=repo,branch="BRANCH_NAME", access_config=GitAccessConfig(token)
        ),
        writer=writer,
        writer_kwargs={},
    )

    runner.run()

## Let's Chat with our ingested github docs in less than 4 lines of code.

In [23]:
import os
#Replace your PAT
os.environ['CLARIFAI_PAT'] = "YOUR_CLARIFAI_PAT"

In [ ]:
from clarifai.rag import RAG
rag_agent = RAG.setup(app_url="YOUR CLARIFAI APP URL",
                      llm_url = "https://clarifai.com/mistralai/completion/models/mistral-large")

### Start chatting with your ingested data.

In [ ]:
result=rag_agent.chat(messages=[{"role":"human", "content":"how to upload dataset using clarifai"}])
(result[0]["content"].split('\n'))